## Chat Models - Transformation Chain + Sequential Chain


In [ ]:
import os
%pip install langchain
os.environ['OPENAI_API_KEY'] = 'API_KEY_HERE'

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain, TransformChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage

In [3]:
# Defining a chat model:
chat = ChatOpenAI(temperature=0.7)

In [6]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    # Extract the last message from the text
    last_message = text.split("\n")[-1]
    return {"output_text": last_message}

transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)

In [7]:
result = transform_chain({'text': 'Hello, how are you?\nI am fine, thanks for asking!\nWhat are you doing today?\nI am working on a new project.'
})
print(result)

{'text': 'Hello, how are you?\nI am fine, thanks for asking!\nWhat are you doing today?\nI am working on a new project.', 'output_text': 'I am working on a new project.'}


------------------------------------------------------------------------

In [19]:
import re

def extract_stock_metrics(inputs: dict) -> dict:
    text = inputs["text"]
    
    # Regular expressions to extract stock market metrics
    opening_price_pattern = r"Opening Price:\s*\$(\d+\.\d+)"
    closing_price_pattern = r"Closing Price:\s*\$(\d+\.\d+)"
    volume_pattern = r"Volume:\s*(\d+)"
    high_pattern = r"High:\s*\$(\d+\.\d+)"
    low_pattern = r"Low:\s*\$(\d+\.\d+)"
    
    # Extract metrics using the regex patterns
    opening_price = re.search(opening_price_pattern, text)
    closing_price = re.search(closing_price_pattern, text)
    volume = re.search(volume_pattern, text)
    high = re.search(high_pattern, text)
    low = re.search(low_pattern, text)

    # If the regex search found the metric, use it, otherwise default to None
    opening_price = float(opening_price.group(1)) if opening_price else None
    closing_price = float(closing_price.group(1)) if closing_price else None
    volume = int(volume.group(1)) if volume else None
    high = float(high.group(1)) if high else None
    low = float(low.group(1)) if low else None
    
    return {
        "opening_price": opening_price,
        "closing_price": closing_price,
        "volume": volume,
        "high": high,
        "low": low
    }


transform_chain = TransformChain(
    input_variables=["text"], 
    output_variables=["opening_price", "closing_price", "volume", "high", "low"],
    transform=extract_stock_metrics
)


In [20]:
transform_chain({
    'text': '''
Stock Update: TechCorp Inc.
Date: September 28, 2023
Opening Price: $142.50
Closing Price: $145.20
Volume: 3200000
High: $146.00
Low: $141.70

TechCorp Inc. experienced a slight increase in their stock price today, closing at $145.20. With a trading volume of 3.2 million, the stock hit a high of $146.00 and a low of $141.70 throughout the trading session. Analysts remain optimistic about the company's future prospects.
'''
})

{'text': "\nStock Update: TechCorp Inc.\nDate: September 28, 2023\nOpening Price: $142.50\nClosing Price: $145.20\nVolume: 3200000\nHigh: $146.00\nLow: $141.70\n\nTechCorp Inc. experienced a slight increase in their stock price today, closing at $145.20. With a trading volume of 3.2 million, the stock hit a high of $146.00 and a low of $141.70 throughout the trading session. Analysts remain optimistic about the company's future prospects.\n",
 'opening_price': 142.5,
 'closing_price': 145.2,
 'volume': 3200000,
 'high': 146.0,
 'low': 141.7}

In [18]:
extract_metrics_template = """
Please extract the key stock metrics from the following update: {text}
"""

system_message_prompt_extract = SystemMessagePromptTemplate(
    prompt=PromptTemplate(template=extract_metrics_template, input_variables=["text"]),
)
extract_metrics_prompt = ChatPromptTemplate.from_messages([system_message_prompt_extract])

# Part 1 Create the chain:
extract_metrics_chain = TransformChain(llm=chat, input_variables=["text"], 
                                       prompt=extract_metrics_prompt, output_variables=["opening_price", "closing_price", "volume", "high", "low"],
                                       transform=extract_stock_metrics)


analysis_template = """
Based on the following stock metrics:
Opening Price: {opening_price}
Closing Price: {closing_price}
Volume: {volume}
High: {high}
Low: {low}

Provide an analysis or commentary on the stock's performance.
"""

extract_metrics_prompt = ChatPromptTemplate.from_messages([HumanMessagePromptTemplate.from_template(analysis_template)])
analysis_chain = LLMChain(llm=chat, prompt=extract_metrics_prompt, output_key="commentary")

# Create the Sequential Chain:
stock_chain = SequentialChain(
    chains=[extract_metrics_chain, analysis_chain],
    input_variables=["text"],
    output_variables=["commentary"],
)

# Run the chain:
result = stock_chain(
    {
        "text": """
        Stock Update: TechCorp Inc.
        
        Date: September 28, 2023
        Opening Price: $142.50
        Closing Price: $145.20
        Volume: 3200000
        High: $146.00
        Low: $141.70
        """
    }
)

print(result)  # This should give a generated analysis or commentary on the stock's performance.

{'text': '\n        Stock Update: TechCorp Inc.\n        \n        Date: September 28, 2023\n        Opening Price: $142.50\n        Closing Price: $145.20\n        Volume: 3200000\n        High: $146.00\n        Low: $141.70\n        ', 'commentary': "The stock experienced a positive performance today, with the opening price of 142.5 increasing to 145.2 by the end of the trading day. This indicates a gain of 2.7 points or approximately 1.89%. The stock also reached a high of 146.0, suggesting potential bullish momentum during the trading session.\n\nThe volume of 3,200,000 indicates a considerable level of market interest and activity in the stock. Higher volume generally signifies increased buying and selling pressure, which can be a positive sign for the stock's performance.\n\nThe stock's low of 141.7 indicates that it did not experience significant downward pressure during the day, as it stayed relatively close to its opening price. This could be seen as a positive sign for invest

----------------------------

In [23]:
# http://api.open-notify.org/astros.json
import requests

def gender_count_transform(inputs: dict) -> dict:
    # Fetch data from an API:
    response = requests.get(url=inputs['url'])
    data = response.json()

    # Ask ChatGPT to classify the males vs females:
    chat = ChatOpenAI()
    resp = chat([SystemMessage(content=f'''You are responsible for classifying all of the people within a JSON request.
                        An good response should be:
                        - Males: 5
                        - Females: 10
                        {data}
                        ----
                        Only return the output of the classification, not the input.
                        I.e:
                        - Males: 2
                        - Females 3
                        ''')])
    return {"classifications": resp.content }


# Test the function
transform_chain = TransformChain(
    input_variables=["url"], output_variables=["classifications"], transform=gender_count_transform
)

transform_chain({
    "url": "http://api.open-notify.org/astros.json"
})

{'url': 'http://api.open-notify.org/astros.json',
 'classifications': "{'Males': 5, 'Females': 8}"}

In [33]:
# Analysis function based on gender count:
def gender_analysis_transform(inputs: dict) -> dict:
    classifications = inputs["classifications"]
    # Parse counts (assuming format "- Males: 2\n- Females: 3")
    male_count = int(classifications.split("Males: ")[1].split("\n")[0])
    female_count = int(classifications.split("Females: ")[1].split("\n")[0])

    if male_count > female_count:
        commentary = "There are more males than females among the astronauts."
    elif female_count > male_count:
        commentary = "There are more females than males among the astronauts."
    else:
        commentary = "The number of male and female astronauts is equal."

    return {"commentary": commentary}


# Create Transform Chains:
gender_count_chain = TransformChain(
    input_variables=["url"], output_variables=["classifications"], transform=gender_count_transform
)

gender_analysis_chain = TransformChain(
    input_variables=["classifications"], output_variables=["commentary"], transform=gender_analysis_transform
)


# Create the Sequential Chain:
astronaut_chain = SequentialChain(
    chains=[gender_count_chain, gender_analysis_chain],
    input_variables=["url"],
    output_variables=["commentary", "classifications"],
)

# Run the chain:
result = astronaut_chain(
    {
        "url": "http://api.open-notify.org/astros.json"
    }
)

print(result["commentary"])

There are more females than males among the astronauts.
